
###Automated stock trading using FinRL

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:

Action: The action space describes the allowed actions that the agent interacts with the environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s', i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively

State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so our trading agent observes many different features to better learn in an interactive environment.

Environment: Dow 30 consituents

Install all the packages through FinRL library

In [1]:
!python --version



Python 3.11.11


In [2]:
!python -m pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-22ikopmb
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-22ikopmb
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 8cf3cacc6f570d26b430e403ea522c8fe9e6876a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-aomqx3rz/elegantrl_7741e2a78cfd4f7388da0a2621960940
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-aomqx3rz/elegantrl_7741e2a78cfd4f7388da0a2621960940
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 3bdc958c8e624b61a9e661832b01fef816924f61
  Preparing metadata (setup.py) ... done


In [3]:
!pip3 install pandas
!pip install numpy

In [4]:
!set PATH=%PATH%;C:\Users\natna\miniforge3\envs\finRl\Scripts


/bin/bash: line 1: C:Usersnatnaminiforge3envsfinRlScripts: command not found


In [5]:
!python -m pip install numpy==1.26.4 scipy==1.12.0 scikit-learn==1.6.1



# !python --version 1.23

In [6]:
import numpy
import scipy
import sklearn
print("Numpy version:", numpy.__version__)
print("Scipy version:", scipy.__version__)
print("Scikit-learn version:", sklearn.__version__)


Numpy version: 1.26.4
Scipy version: 1.12.0
Scikit-learn version: 1.6.1


Import Packages

In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl import config
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/home/aboba/anaconda3/envs/finrl_env/lib/python3.11/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [8]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


Create Folders

In [9]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

Download Data

In [10]:
from finrl import config_tickers
df = YahooDownloader(start_date = '2009-01-01',
                           end_date = '2020-09-30',
                           ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (86111, 8)


Add technical Indicators

In [11]:
df = FeatureEngineer(use_technical_indicator=True,
                      tech_indicator_list = config.INDICATORS,
                      use_turbulence=True,
                      user_defined_feature = False).preprocess_data(df.copy())

Successfully added technical indicators
Successfully added turbulence index


In [12]:
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values
  cov_list.append(covs)

df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
df.head()

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,cov_list
0,2010-01-04,6.440332,7.660714,7.585000,7.622500,493729600,AAPL,0,0.118159,6.503820,5.537622,62.133204,168.772520,33.760635,6.025190,5.972406,0.0,"[[0.000443059449380718, 0.00013698037579451913..."
1,2010-01-04,40.265976,57.869999,56.560001,56.630001,5277400,AMGN,0,0.224876,40.696417,38.247783,52.849978,85.532074,6.350919,39.465821,39.391293,0.0,"[[0.000443059449380718, 0.00013698037579451913..."
2,2010-01-04,32.828991,41.099998,40.389999,40.810001,6894300,AXP,0,0.288681,33.751084,31.445586,56.779361,1.010637,11.537387,32.716404,31.134858,0.0,"[[0.000443059449380718, 0.00013698037579451913..."
3,2010-01-04,43.777550,56.389999,54.799999,55.720001,6186700,BA,0,0.501041,44.009583,41.898210,58.805022,81.205372,10.840906,42.319078,40.753000,0.0,"[[0.000443059449380718, 0.00013698037579451913..."
4,2010-01-04,39.738228,59.189999,57.509998,57.650002,7325600,CAT,0,0.075448,40.134157,38.238728,55.292739,49.350630,8.534279,39.345487,38.969693,0.0,"[[0.000443059449380718, 0.00013698037579451913..."


In real life trading, the model needs to be updated periodically using rolling windows. But here I'm just cutting the data into train and trade set.

In [13]:
train = data_split(df, '2009-01-01','2019-12-31')
trade = data_split(df, '2020-01-01','2020-09-30')

State Space and Action Space Calculation

In [14]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension

In [15]:
print(stock_dimension)
print(state_space)

29
291


## Environment Details

In [16]:
# Define transaction cost lists for buying and selling stocks
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
# Explanation: 
# - `buy_cost_list` and `sell_cost_list` represent the transaction costs as a percentage for buying and selling stocks.
# - `[0.001] * stock_dimension` creates a list where each element is 0.001 (0.1% transaction fee), repeated for each stock.
# - The use of `=` assigns the same list to both `buy_cost_list` and `sell_cost_list`.

# Initialize the list to track the number of shares owned for each stock
num_stock_shares = [0] * stock_dimension
# Explanation:
# - `num_stock_shares` is a list where each element is initialized to 0, representing that no shares are owned initially.
# - `[0] * stock_dimension` ensures the list length matches the number of stocks (`stock_dimension`).

# Create a dictionary to store environment configuration parameters
env_kwargs = {
    "hmax": 100,  # Maximum number of shares that can be bought or sold in a single transaction.
    "initial_amount": 1_000_000,  # Initial cash available for the agent to trade with (e.g., $1,000,000).
    "num_stock_shares": num_stock_shares,  # Initial portfolio: number of shares owned for each stock.
    "buy_cost_pct": buy_cost_list,  # Transaction cost percentage for buying stocks.
    "sell_cost_pct": sell_cost_list,  # Transaction cost percentage for selling stocks.
    "state_space": state_space,  # Dimension of the state space (e.g., features describing the environment).
    "stock_dim": stock_dimension,  # Number of stocks being traded (dimension of the stock universe).
    "tech_indicator_list": config.INDICATORS,  # List of technical indicators used as features for the state space.
    "action_space": stock_dimension,  # Dimension of the action space (one action per stock).
    "reward_scaling": 1e-4  # Scaling factor for rewards to normalize them and improve learning stability.
}
# Explanation:
# - This dictionary (`env_kwargs`) encapsulates all the necessary parameters required to initialize the stock trading environment.
# - It includes configuration for portfolio management (e.g., `hmax`, `initial_amount`, `num_stock_shares`) and the structure of the RL problem (e.g., `state_space`, `action_space`).

# Initialize the stock trading environment with the training data and configuration parameters
e_train_gym = StockTradingEnv(df=train, **env_kwargs)
# Explanation:
# - `StockTradingEnv` is a custom environment class for stock trading, compliant with OpenAI Gym standards.
# - `df=train` specifies the training data (a DataFrame containing historical stock prices and other features).
# - `**env_kwargs` unpacks the `env_kwargs` dictionary, passing each key-value pair as an argument to the environment initializer.
# - The environment simulates the stock trading process, enabling the RL agent to interact with it by observing states, taking actions, and receiving rewards.


Environment for training

In [17]:
env_train, _ = e_train_gym.get_sb_env() #get stable baseline environment for training
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [18]:
agent = DRLAgent(env = env_train)
# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = False
if_using_sac = False

In [19]:
from stable_baselines3.common.logger import configure

 Implement DRL Algorithms

In [20]:
import torch

print('Current version of PyTorch: ', torch.__version__)

if torch.cuda.is_available:
  print('PyTorch can use GPUs!')
else:
  print('PyTorch cannot use GPUs.')

Current version of PyTorch:  2.6.0+cu124
PyTorch can use GPUs!


DDPG

Training

In [21]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = config.RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device
Logging to results/ddpg


In [22]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 92         |
|    time_elapsed    | 108        |
|    total_timesteps | 10060      |
| train/             |            |
|    actor_loss      | 101        |
|    critic_loss     | 2.8e+03    |
|    learning_rate   | 0.001      |
|    n_updates       | 9959       |
|    reward          | -1.6278315 |
-----------------------------------
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 93         |
|    time_elapsed    | 215        |
|    total_timesteps | 20120      |
| train/             |            |
|    actor_loss      | 55.4       |
|    critic_loss     | 2.03       |
|    learning_rate   | 0.001      |
|    n_updates       | 20019      |
|    reward          | -1.6278315 |
-----------------------------------
day: 2514, episode: 10
begin_total_asset: 1000000.00
end_total_a

In [23]:
trained_ddpg.save(config.TRAINED_MODEL_DIR + "/agent_ddpg") if if_using_ddpg else None


Trading

In [24]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [25]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ddpg,
    environment = e_trade_gym)

hit end!


In [26]:
df_account_value.tail()

,date,account_value
183,2020-09-23,809889.780316
184,2020-09-24,809881.904049
185,2020-09-25,822188.690014
186,2020-09-28,840397.895657
187,2020-09-29,833575.681423


Backtesting Performance

In [27]:
df_dji = YahooDownloader(
    start_date='2020-01-01', end_date='2020-09-30', ticker_list=["dji"]
).fetch_data()
df_dji = df_dji[["date", "close"]]
fst_day = df_dji["close"][0]
dji = pd.merge(
    df_dji["date"],
    df_dji["close"].div(fst_day).mul(1000000),
    how="outer",
    left_index=True,
    right_index=True,
).set_index("date")


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (183, 8)


In [28]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         -0.216511
Cumulative returns    -0.166424
Annual volatility      0.479519
Sharpe ratio          -0.270607
Calmar ratio          -0.496245
Stability              0.059184
Max drawdown          -0.436300
Omega ratio            0.947835
Sortino ratio         -0.369223
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.956788
Daily value at risk   -0.060929
dtype: float64


In [29]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI",
        start = '2020-01-01',
        end = '2020-09-30')

stats = backtest_stats(baseline_df, value_col_name = 'close')

==============Get Baseline Stats===========


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (188, 8)
Annual return         -0.065199
Cumulative returns    -0.049054
Annual volatility      0.416030
Sharpe ratio           0.046016
Calmar ratio          -0.175803
Stability              0.012240
Max drawdown          -0.370862
Omega ratio            1.009343
Sortino ratio          0.062829
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.860019
Daily value at risk   -0.052339
dtype: float64


Back Test Plot

In [30]:
df_result_ddpg = df_account_value.set_index(df_account_value.columns[0])
result = pd.DataFrame(
    {
        "ddpg": df_result_ddpg["account_value"],
        "dji": dji["close"],
    }
)
result

,ddpg,dji
date,,
2020-01-02,1000000.000000,NaN
2020-01-03,997987.774266,1.000000e+06
2020-01-06,998669.021350,1.002392e+06
2020-01-07,996850.429082,9.982119e+05
2020-01-08,997018.938846,1.003848e+06
...,...,...
2020-09-23,809889.780316,9.346322e+05
2020-09-24,809881.904049,9.364587e+05
2020-09-25,822188.690014,9.489818e+05


In [31]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
result.plot()
plt.savefig('results_ddpg.png')

PPO

In [32]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = config.RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cuda device
Logging to results/ppo


/home/aboba/anaconda3/envs/finrl_env/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [33]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=200000) if if_using_ppo else None

----------------------------------
| time/              |           |
|    fps             | 174       |
|    iterations      | 1         |
|    time_elapsed    | 11        |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 0.5651754 |
----------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 170         |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.0175762   |
|    clip_fraction        | 0.219       |
|    clip_range           | 0.2         |
|    entropy_loss         | -41.2       |
|    explained_variance   | -0.0143     |
|    learning_rate        | 0.00025     |
|    loss                 | 5.71        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0266     |
|    reward  

In [34]:
trained_ppo.save(config.TRAINED_MODEL_DIR + "/agent_ppo") if if_using_ppo else None

Trading

In [35]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [36]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ppo,
    environment = e_trade_gym)

hit end!


In [37]:
df_account_value.tail()

,date,account_value
183,2020-09-23,900814.247640
184,2020-09-24,906593.959866
185,2020-09-25,919445.367209
186,2020-09-28,932056.240344
187,2020-09-29,927069.374144


Backtesting Performance

In [38]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         -0.096524
Cumulative returns    -0.072931
Annual volatility      0.426166
Sharpe ratio          -0.026455
Calmar ratio          -0.276040
Stability              0.005532
Max drawdown          -0.349675
Omega ratio            0.994663
Sortino ratio         -0.036719
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.857310
Daily value at risk   -0.053737
dtype: float64


In [39]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI",
        start = '2020-01-01',
        end = '2020-09-30')

stats = backtest_stats(baseline_df, value_col_name = 'close')

[*********************100%***********************]  1 of 1 completed

==============Get Baseline Stats===========
Shape of DataFrame:  (188, 8)
Annual return         -0.065199
Cumulative returns    -0.049054
Annual volatility      0.416030
Sharpe ratio           0.046016
Calmar ratio          -0.175803
Stability              0.012240
Max drawdown          -0.370862
Omega ratio            1.009343
Sortino ratio          0.062829
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.860019
Daily value at risk   -0.052339
dtype: float64


In [40]:
df_result_ppo = df_account_value.set_index(df_account_value.columns[0])
result = pd.DataFrame(
    {
        "ppo": df_result_ppo["account_value"],
        "dji": dji["close"],
    }
)
result

,ppo,dji
date,,
2020-01-02,1.000000e+06,NaN
2020-01-03,9.993839e+05,1.000000e+06
2020-01-06,9.997275e+05,1.002392e+06
2020-01-07,9.990258e+05,9.982119e+05
2020-01-08,1.000956e+06,1.003848e+06
...,...,...
2020-09-23,9.008142e+05,9.346322e+05
2020-09-24,9.065940e+05,9.364587e+05
2020-09-25,9.194454e+05,9.489818e+05


In [41]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
result.plot()

plt.savefig('results_ppo.png')

A2C

In [42]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = config.RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
Logging to results/a2c


/home/aboba/anaconda3/envs/finrl_env/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [43]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

--------------------------------------
| time/                 |            |
|    fps                | 139        |
|    iterations         | 100        |
|    time_elapsed       | 3          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -41.4      |
|    explained_variance | -0.219     |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -83.6      |
|    reward             | -0.2211089 |
|    std                | 1.01       |
|    value_loss         | 5.52       |
--------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 140       |
|    iterations         | 200       |
|    time_elapsed       | 7         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -41.5     |
|    explained_variance | -0.0862   |
|    learning_rate      | 0.0007  

In [44]:
trained_a2c.save(config.TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None

Trading

In [45]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [46]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment = e_trade_gym)

hit end!


In [47]:
df_account_value.tail()

,date,account_value
183,2020-09-23,1.012342e+06
184,2020-09-24,1.015620e+06
185,2020-09-25,1.022471e+06
186,2020-09-28,1.032990e+06
187,2020-09-29,1.026637e+06


Backtesting Performance

In [48]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.035866
Cumulative returns     0.026637
Annual volatility      0.366240
Sharpe ratio           0.279592
Calmar ratio           0.116998
Stability              0.087663
Max drawdown          -0.306548
Omega ratio            1.057756
Sortino ratio          0.392271
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.998670
Daily value at risk   -0.045736
dtype: float64


In [49]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI",
        start = '2020-01-01',
        end = '2020-09-30')

stats = backtest_stats(baseline_df, value_col_name = 'close')

[*********************100%***********************]  1 of 1 completed

==============Get Baseline Stats===========
Shape of DataFrame:  (188, 8)
Annual return         -0.065199
Cumulative returns    -0.049054
Annual volatility      0.416030
Sharpe ratio           0.046016
Calmar ratio          -0.175803
Stability              0.012240
Max drawdown          -0.370862
Omega ratio            1.009343
Sortino ratio          0.062829
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.860019
Daily value at risk   -0.052339
dtype: float64


In [50]:
df_result_a2c = df_account_value.set_index(df_account_value.columns[0])
result = pd.DataFrame(
    {
        "a2c": df_result_a2c["account_value"],
        "dji": dji["close"],
    }
)
result

,a2c,dji
date,,
2020-01-02,1.000000e+06,NaN
2020-01-03,9.988238e+05,1.000000e+06
2020-01-06,9.993823e+05,1.002392e+06
2020-01-07,9.984936e+05,9.982119e+05
2020-01-08,1.001921e+06,1.003848e+06
...,...,...
2020-09-23,1.012342e+06,9.346322e+05
2020-09-24,1.015620e+06,9.364587e+05
2020-09-25,1.022471e+06,9.489818e+05


In [51]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
plt.show()

/tmp/ipykernel_45767/4043962701.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [52]:
result.plot()
plt.savefig('results_a2c.png')